In [ ]:
import numpy as np
import scipy as sp
import math
from scipy import optimize
import sympy as sm
from scipy import linalg
from sympy import Derivative
from scipy import interpolate
import scipy.integrate as quad
from scipy.integrate import odeint
import scipy.special as special
from scipy.integrate import quad
from sympy import symbol,function
import ipywidgets as widgets

#autoreload model when code is run
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

## Consumer optimization problem

Before moving to the Ak model adding the firm player to our model, we focus on consumers and their optimization problem. The standard Ak model assumes that the individual show a CRRA utility function with respect to consumption of the type:  

$$u(c)=\frac{c^{1-\theta}-1}{1-\theta}$$  

where $\theta$ is a measure or relative risk aversion
This type of preferences is really useful in the case $\theta=1$ because in that occasion the CRRA utility becomes a logarithmic utility function.  
Lets create a simple utility maximization problem with two periods to see how the above preferences impact the consumer behavior. We take into consideration two periods, $t=0$ and $t=1$.   
Our representative consumer maximizes consumption in the present $c_0$ and in the future $c_1$ taking into consideration his labour income will be $w_0$ today and $w_1$ tomorrow. So, our utility maximization problem become:  

$$max U(c_0,c_1) = u(c_0)+e^{-rho}u(c_1)$$  

under the constraint:  

$$c_0+\frac{1}{1+r}{c_1} = w_0 +\frac{1}{1+r}{w_1}$$ 
where r is the interest rate.

Now, before solving the problem, we have to give some initial values:


In [ ]:
#a.define initial parameters


#b.define utility function
def utility(ct):
    return (ct**(1-theta)-1)/(1-theta)

We set the lagrangian:  

$$L=\max_{c_{0},c_{2}}\bigg(\frac{c_{0}^{1 - \theta} - 1}{1 - \theta}\bigg) + e^{-\rho}\bigg(\frac{c_{1}^{1 - \theta} - 1}{1 - \theta}\bigg) + \lambda\bigg(w_{0} + \frac{1}{1 + r}w_{1} - c_{0} - \frac{1}{1 + r}c_{1}\bigg)$$  

Focs are:
$$c_{0}^{-\theta} = \lambda$$
$$e^{-\rho}c_{1}^{-\theta}=\frac{1}{1 + r}\lambda$$  

The above can reduce to:
$$c_{0}^{-\theta} = e^{-\rho}(1 + r)c_{1}^{-\theta}$$  

We can now start writing our code:

In [ ]:
def util(c,theta):
    return (c**(1-theta)-1)/(1-theta) #Single period utility

def u_func(c0,c1,rho,theta):
    return util(c0,theta)+np.exp(-rho)*util(c1,theta) #Two period utility


# Intial values
theta = 0.5
w0= 5
w1 = 5
rho = 0.9
r = 0.5

In [ ]:
#define utility function for two periods
def inter_util(c_t, c_t1):
    return utility(c_t)+np.exp(-rho)*utility(c_t1)

#b.define Euler equation
def euler(c_t,c_t1):
    return c_t**(-theta)-np.exp(-rho)*(1+r)*(c_t1)**(-theta)

#c.define constraint
def constraint(c_t,c_t1):
    return w_t+(1/(1+r))*w_t1-c_t-(1/(1+r))*c_t1

#d.creating optimizing function
def optimalchoice(x):
    op = [euler(x[0], x[1])]
    op.append(constraint(x[0],x[1]))
    return op
 
# Apply fsolve()...note that we need to give the algorithm a decent guess!
ct_guess, ct1_guess = 0.5, 0.5
ct_star, ct1_star = optimize.fsolve(optimalchoice, x0=(ct_guess, ct1_guess))
utility_star = inter_util(ct_star, ct1_star)

    

print(f'The optimal consumption at the present is: {ct_star:.2f}')
print(f'The optimal consumption tomorrow is: {ct1_star:.2f}')
print(f'Utility from optimal bundle is: {utility_star:.2f}')

We now construct an interactive plot to study the behaviour of our consumer preferences for different values of main variables.

In [ ]:
def plot_max_problem(w_t, w_t1, theta, rho):

    fig1 = plt.figure(dpi=150)
    ax = fig1.add_subplot(1,1,1)

    
    #grid of (x,y) values which we will pass to function
    ct_range = np.arange(0, 10, 1)
    ct1_range = np.arange(0, 10, 1)
    c_t, c_t1 = np.meshgrid(ct_range, ct1_range)
    
    # we will actually plot utility
    uti = inter_util(c_t, c_t1)

    # plot the budget constraint
    cons_today = np.linspace(0, 10, 100)
    ax.plot(cons_today, (1 + r) * (w_t - cons_today) + w_t1, 
           color='r')

    # demarcate the indifference curve
    CS = ax.contour(ct_star, ct1_star, utility_star, np.array([utility_star]), colors='k', linewidths=1, linestyles='solid')
    ax.clabel(CS, inline=1, fmt='%1.4f')

    # mark the optimal bundle
    ax.hlines(ct_star, 0, ct1_star, linestyle='dashed')
    ax.vlines(ct1_star, 0, ct_star, linestyle='dashed')

    # axes, labels, title, colorbar etc.
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.set_xlabel(r'Present Consumption, $C_{t}$', fontsize=10)
    ax.set_ylabel(r'Future Consumption, $C_{t+1}$', fontsize=10)
    ax.set_title(r'Optimal bundle for CRRA utility', fontsize=15)
    plt.show()

def plot_time():
    widgets.interact(plot_max_problem,
    w_t= widgets.FloatSlider(
           description='$w_{t}$',
           min=0,
           max=10,
           step=1,
           value=5,
           continuous_update=False,
    ),
    
    w_t1 = widgets.FloatSlider(
            description="$w_{t+1}$",
            min=0,
            max=10,
            step=1,
            value=5,
            continuous_update=False,
    ),
    theta = widgets.FloatSlider(
            description="$\\theta$",
            min=0.5,
            max=0.9,
            step=0.02,
            value=0.5,
            continuous_update=False,
    ),
    rho = widgets.FloatSlider(
            description="$\\rho$",
            min=0.9,
            max=0.99,
            step=0.01,
            value=0.9,
            continuous_update=False,
    ),
);  
    